In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pip install sentence-transformers

In [ ]:
import numpy as np
import pandas as pd
import sentence_transformers
from sentence_transformers import SentenceTransformer,models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from sentence_transformers.losses import DenoisingAutoEncoderLoss
import torch
from torch.utils.data import DataLoader
import re

In [ ]:
df = pd.read_csv('/kaggle/input/job-data/job_data.csv')
df = df.dropna()
df.head(1)

In [ ]:
sentences = []
def generate_sentence(jd):
    spliter = re.compile(r'\.\s?\n?')
    list_of_sentences = spliter.split(jd)
    if len(sentences)<100_000:
        sentences.extend([i for i in list_of_sentences if len(i)>30])
df.jd.apply(generate_sentence)
len(sentences)

In [ ]:
def clean_sentence(text):
    text = text.lower()
    text = re.sub("[^ A-Za-z0-9.&,\-]"," ",text)
    text = re.sub(' +',' ',text)
    return text
sentences = [clean_sentence(i) for i in sentences]

In [ ]:
sentences[:3]

In [ ]:
# sentences = sentences[:50_000]

In [ ]:
# The DenoisingAutoEncoderDataset returns InputExamples in the format: texts=[noise_fn(sentence), sentence] 
# add noise in traning data
train_data = DenoisingAutoEncoderDataset(sentences)
loader = DataLoader(train_data,batch_size=4,shuffle=True)

In [ ]:
# Original sentence
sentences[0]

In [ ]:
# in traning data we delete remove word
train_data[0].texts # [noise_senrence, Original_sentence]

In [ ]:
gte_model = models.Transformer('thenlper/gte-base')
polling = models.Pooling(gte_model.get_word_embedding_dimension(),'cls')
model = SentenceTransformer(modules = [gte_model,polling])

In [ ]:
loss = DenoisingAutoEncoderLoss(model,tie_encoder_decoder = True)

**Train Model**

In [ ]:
model.fit([(loader,loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

In [ ]:
model.save('output/gte-base-fine-tune')

**Validate Model**